<a href="https://colab.research.google.com/github/erico-bachour/Fitness_Project/blob/main/Fitness_Project_2_An%C3%A1lise_e_Regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Fitness Project - Erico Bachour**

## Análise dos Dados e Regressão

Para entender melhor a jornada, leia o README em minha página no GitHub (github.com/erico-bachour).

Nesse notebook, o objetivo será importar a base de dados feita no notebook 1 e adicionar algumas colunas, que gosto de considerar como 'analíticas'. Além disso, faremos uma análise dos dados iniciais, junto com as novas colunas. Numa última etapa, faremos uma regressão linear para responder a seguinte pergunta:

"Estar em déficit calórico significa mesmo perder peso ou gordura?"

##1. Visualização do dataset

In [18]:
import pandas as pd

In [19]:
dataset = pd.read_csv('/content/drive/MyDrive/Fitness/dataset_fitness.csv')

In [20]:
dataset.drop(columns='Unnamed: 0',axis=1, inplace=True)

In [21]:
dataset.tail()

,data,%Gordura,Massa Magra (kg),Peso (kg),Calorias Consumidas (kcal),Carboidratos (g),Gorduras (g),Proteinas (g),Calorias Gastas (kcal),Passos Dados,Calorias Exercicio (kcal)
478,2023-03-02,16.3,59.7,77.80,2727.0,180.0,92.0,153.0,1223.0,17212.0,0.0
479,2023-03-03,16.1,59.2,76.95,2553.0,195.0,94.0,155.0,1123.0,13852.0,0.0
480,2023-03-04,15.7,59.1,76.40,4072.0,453.0,95.0,153.0,1028.0,9567.0,0.0
481,2023-03-05,17.1,59.8,78.80,2805.0,241.0,96.0,145.0,862.0,11187.0,0.0
482,2023-03-06,16.6,59.2,77.55,2275.0,197.0,72.0,196.0,0.0,NaN,0.0


In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   data                        483 non-null    object 
 1   %Gordura                    446 non-null    float64
 2   Massa Magra (kg)            446 non-null    float64
 3   Peso (kg)                   480 non-null    float64
 4   Calorias Consumidas (kcal)  483 non-null    float64
 5   Carboidratos (g)            251 non-null    float64
 6   Gorduras (g)                251 non-null    float64
 7   Proteinas (g)               251 non-null    float64
 8   Calorias Gastas (kcal)      483 non-null    float64
 9   Passos Dados                203 non-null    float64
 10  Calorias Exercicio (kcal)   483 non-null    float64
dtypes: float64(10), object(1)
memory usage: 41.6+ KB


##2. Alterações e adições ao dataset

###2.1. Convertendo coluna 'data' em 'datetime'

In [23]:
dataset['data'] = pd.to_datetime(dataset['data'])

In [24]:
dataset.data

0     2021-10-25
1     2021-10-26
2     2021-10-27
3     2021-10-28
4     2021-10-29
         ...    
478   2023-03-02
479   2023-03-03
480   2023-03-04
481   2023-03-05
482   2023-03-06
Name: data, Length: 483, dtype: datetime64[ns]

###2.2. Criação de colunas analíticas

####2.2.1. Variações

In [25]:
def calcular_variacao(row, dataset):
  # Se a linha atual é a primeira linha, defina a variação como 0 
  if row.name == dataset.index[0]:
    row['Var_Peso_(kg)'] = 0
    row['Var_%Gordura'] = 0
    row['Var_Massa_Magra_(kg)'] = 0
  else:
    # Calcular a variação das colunas primárias e armazenar em uma nova coluna chamada 'Var_'
    row['Var_Peso_(kg)'] = row['Peso (kg)'] - dataset.loc[row.name-1, 'Peso (kg)']
    row['Var_%Gordura'] = row['%Gordura'] - dataset.loc[row.name-1, '%Gordura']
    row['Var_Massa_Magra_(kg)'] = row['Massa Magra (kg)'] - dataset.loc[row.name-1, 'Massa Magra (kg)']
  return row

# Criar nova coluna chamada 'Var_', aplicando a função calcular_variacao() para cada linha do DataFrame
dataset = dataset.apply(calcular_variacao, axis=1, args=(dataset,))

####2.2.2. Balanço Energético

A teoria nos diz que a base para o ganho ou perda de peso é o balanço energético, ou seja, se o indvíduo está ingerindo mais ou menos calorias do que ele gasta.

Esse balanço é definido pela seguinte equação:

Balanço Energético = Alimentação - (TMB + Exercícios)

 - TMB: Taxa de Metabolismo Basal, o mínimo de calorias que o seu organismo precisa para manter-se funcionando.

Para obtermos perda de peso ou perda de gordura, é importante mantermos uma constância de valores negativos no Balanço Energético. Do contrário, teremos o processo inverso, com aumento do peso e do percentual de gordura.

O objetivo no processo de "cutting" é perder o máximo de percentual de gordura mantendo massa magra. Já no processo de "bulking" a meta é ganhar o máximo de massa magra, aumentando o mínimo possivel o percentual de gordura. Assim, no "cutting" a meta é manter um balanço energético negativo, enquanto no "bulking", o objetivo é um balanço energético positivo.

#### Equação de Harris-Benedict

Para o cálculo da TMB, utilizaremos como base a equação de Harris-Benedict (revista por Mifflin e St. Jeor, em 1990)

**TMB = (10 × peso em kg) + (6.25 × altura em cm) - (5 × idade em anos) + 5**

In [26]:
from datetime import datetime

dob = datetime(1990,11,12)

In [27]:
dataset['TMB'] = ((dataset['Peso (kg)'] * 10) + (6.25 * 174) - 5 * ((dataset['data'] - dob).dt.days / 365.25) + 5).round(0)

#### Gasto Calórico

No gasto calórico estão incluídas as calorias gastas naturalmente com o funcionamento do corpo somadas às calorias gastas em exercícios.

In [28]:
dataset['Gasto_calorico'] = dataset['TMB'] + dataset['Calorias Gastas (kcal)'] + dataset['Calorias Exercicio (kcal)']

In [29]:
dataset['Balanco_energetico'] = dataset['Calorias Consumidas (kcal)'] - dataset['Gasto_calorico']

####2.2.3. Média Móvel de peso

Linha de tendência da variável 'Peso (kg)'

Para evitar falhas na média móvel vamos completar as falhas na medição do peso.

In [30]:
dataset["Peso (kg)"].fillna(method='ffill', inplace=True)

In [31]:
dataset['Media_Movel'] = dataset['Peso (kg)'].rolling(5).mean()

In [32]:
import datetime
dataset[(dataset['data'].dt.month == 2) & (dataset['data'].dt.year == 2023)]

,data,%Gordura,Massa Magra (kg),Peso (kg),Calorias Consumidas (kcal),Carboidratos (g),Gorduras (g),Proteinas (g),Calorias Gastas (kcal),Passos Dados,Calorias Exercicio (kcal),Var_Peso_(kg),Var_%Gordura,Var_Massa_Magra_(kg),TMB,Gasto_calorico,Balanco_energetico,Media_Movel
449,2023-02-01,15.7,58.4,75.55,2921.0,194.0,128.0,187.0,1056.0,16538.0,0.0,-0.20,-0.1,-0.1,1687.0,2743.0,178.0,75.54
450,2023-02-02,15.3,58.4,75.15,2468.0,244.0,78.0,190.0,772.0,10838.0,0.0,-0.40,-0.4,0.0,1683.0,2455.0,13.0,75.41
451,2023-02-03,15.6,58.4,75.40,2593.0,237.0,93.0,195.0,690.0,11278.0,0.0,0.25,0.3,0.0,1685.0,2375.0,218.0,75.52
452,2023-02-04,16.3,57.8,75.30,3508.0,203.0,126.0,185.0,939.0,16697.0,0.0,-0.10,0.7,-0.6,1684.0,2623.0,885.0,75.43
453,2023-02-05,15.6,58.6,75.70,2926.0,186.0,165.0,176.0,1004.0,14645.0,0.0,0.40,-0.7,0.8,1688.0,2692.0,234.0,75.42
454,2023-02-06,16.2,58.6,76.30,2665.0,235.0,107.0,181.0,1066.0,16784.0,0.0,0.60,0.6,0.0,1694.0,2760.0,-95.0,75.57
455,2023-02-07,16.6,58.4,76.40,3654.0,346.0,188.0,175.0,915.0,15402.0,0.0,0.10,0.4,-0.2,1695.0,2610.0,1044.0,75.82
456,2023-02-08,15.8,59.1,76.50,2403.0,192.0,81.0,219.0,778.0,12817.0,0.0,0.10,-0.8,0.7,1696.0,2474.0,-71.0,76.04
457,2023-02-09,16.3,58.8,76.60,2287.0,167.0,106.0,159.0,1002.0,17299.0,0.0,0.10,0.5,-0.3,1697.0,2699.0,-412.0,76.30
458,2023-02-10,15.7,58.5,75.65,4432.0,400.0,117.0,221.0,856.0,13929.0,0.0,-0.95,-0.6,-0.3,1688.0,2544.0,1888.0,76.29


In [33]:
dataset

,data,%Gordura,Massa Magra (kg),Peso (kg),Calorias Consumidas (kcal),Carboidratos (g),Gorduras (g),Proteinas (g),Calorias Gastas (kcal),Passos Dados,Calorias Exercicio (kcal),Var_Peso_(kg),Var_%Gordura,Var_Massa_Magra_(kg),TMB,Gasto_calorico,Balanco_energetico,Media_Movel
0,2021-10-25,NaN,NaN,91.30,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.00,0.0,0.0,1851.0,1851.0,-1851.0,NaN
1,2021-10-26,NaN,NaN,90.40,0.0,NaN,NaN,NaN,0.0,NaN,0.0,-0.90,NaN,NaN,1842.0,1842.0,-1842.0,NaN
2,2021-10-27,NaN,NaN,89.90,0.0,NaN,NaN,NaN,0.0,NaN,150.0,-0.50,NaN,NaN,1837.0,1987.0,-1987.0,NaN
3,2021-10-28,NaN,NaN,89.50,0.0,NaN,NaN,NaN,0.0,NaN,0.0,-0.40,NaN,NaN,1833.0,1833.0,-1833.0,NaN
4,2021-10-29,NaN,NaN,89.20,0.0,NaN,NaN,NaN,0.0,NaN,0.0,-0.30,NaN,NaN,1830.0,1830.0,-1830.0,90.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,2023-03-02,16.3,59.7,77.80,2727.0,180.0,92.0,153.0,1223.0,17212.0,0.0,0.00,-0.3,0.3,1709.0,2932.0,-205.0,77.49
479,2023-03-03,16.1,59.2,76.95,2553.0,195.0,94.0,155.0,1123.0,13852.0,0.0,-0.85,-0.2,-0.5,1700.0,2823.0,-270.0,77.41
480,2023-03-04,15.7,59.1,76.40,4072.0,453.0,95.0,153.0,1028.0,9567.0,0.0,-0.55,-0.4,-0.1,1695.0,2723.0,1349.0,77.25
481,2023-03-05,17.1,59.8,78.80,2805.0,241.0,96.0,145.0,862.0,11187.0,0.0,2.40,1.4,0.7,1719.0,2581.0,224.0,77.55
